In [1]:
# reference: https://plot.ly/~jackp/17421/plotly-candlestick-chart-in-python/#/ 

import pandas as pd
import numpy as np
import sqlite3
import matplotlib
%matplotlib inline
import plotly
plotly.tools.set_credentials_file(username='drimik99', api_key='Qc102fKlOrNzjG5RZm6B')

In [2]:
import os
os.listdir()

['orders_table.csv',
 'trades_table.csv',
 '3-5',
 'orders_trades_nov11_analysis.ipynb',
 '.ipynb_checkpoints']

In [3]:
df_orders = pd.read_csv('orders_table.csv')
df_trades = pd.read_csv('trades_table.csv')

df_orders_orig = df_orders.copy()
df_trades_orig = df_trades.copy()
# df_trades_kr[(df_trades_kr['pair1']=='BTC')&(df_trades_kr['pair2']=='USD')].plot(x='ts', y='price', figsize=(10,8))

## Choose what Parameters you would like to see:

In [4]:
pair1='ETH'
pair2='BTC'
time_interval='2s'
exchange='GDAX'
# Specify what moving averages you would like to see:

## Let Jadoo do its work

In [5]:
def convert_col_to_datetime(df):
    df['exchangets'] = pd.to_datetime(df['exchangets'])
    df['ts'] = pd.to_datetime(df['ts'])
    return df

In [6]:
# choose pair
# choose varying time interval
# groupby dataframe for the interval -- select the maximum, minimu, starting, ending of the 
    # interval and choose the value to display

# plot the groupby dataframe with x being the interval & y being the value of the price pair

df_trades_gemini = df_trades[df_trades['exchange']==exchange]
df_trades_gemini = df_trades_gemini[(df_trades_gemini['pair1']==pair1) & (df_trades_gemini['pair2']==pair2)]
df_trades_gemini = convert_col_to_datetime(df_trades_gemini)
df_trades_gemini['price_min'] = df_trades_gemini['price']
df_trades_gemini['price_max'] = df_trades_gemini['price']
df_trades_gemini['price_start'] = df_trades_gemini['price']
df_trades_gemini['price_close'] = df_trades_gemini['price']
# df_trades_gemini['price_min'] = df_trades_gemini['price']
# df_trades_gemini['price_min'] = df_trades_gemini['price']



In [7]:
df_trades_gemini.reset_index(drop=True, inplace=True)

In [8]:
df_trades_gemini.sort_values('exchangets', inplace=True)

In [9]:
df_trades_gemini.head()

,exchange,pair1,pair2,price,volume,isbuy,islimit,exchangets,ts,price_min,price_max,price_start,price_close
99,GDAX,ETH,BTC,0.03287,1.072160,True,True,2018-11-11 18:07:44,2018-11-11 18:48:30,0.03287,0.03287,0.03287,0.03287
98,GDAX,ETH,BTC,0.03285,0.010000,True,True,2018-11-11 18:11:59,2018-11-11 18:48:30,0.03285,0.03285,0.03285,0.03285
97,GDAX,ETH,BTC,0.03285,0.263133,True,True,2018-11-11 18:11:59,2018-11-11 18:48:30,0.03285,0.03285,0.03285,0.03285
96,GDAX,ETH,BTC,0.03285,0.454808,True,True,2018-11-11 18:12:48,2018-11-11 18:48:30,0.03285,0.03285,0.03285,0.03285
95,GDAX,ETH,BTC,0.03285,0.349491,True,True,2018-11-11 18:12:49,2018-11-11 18:48:30,0.03285,0.03285,0.03285,0.03285


In [10]:
df_prices_modified = df_trades_gemini.groupby(pd.Grouper(key='exchangets', freq=time_interval)).aggregate({'price':'mean',
                                                                             'volume':'mean',
                                                                            'price_min':'min',
                                                                            'price_max':'max',
                                                                            'price_start':'first',
                                                                            'price_close':'last'})

# df_trades_gemini.groupby(pd.Grouper(key='exchangets', freq='10S')).aggregate({'price':'mean', 
#                                                                               'volume':'mean',
#                                                                              'exchangets':'mean'}).reset_index()

In [11]:
df_prices_modified.dropna(inplace=True)

In [12]:
df_prices_modified.head()

,price,volume,price_min,price_max,price_start,price_close
exchangets,,,,,,
2018-11-11 18:07:44,0.03287,1.072160,0.03287,0.03287,0.03287,0.03287
2018-11-11 18:11:58,0.03285,0.136567,0.03285,0.03285,0.03285,0.03285
2018-11-11 18:12:48,0.03285,0.402149,0.03285,0.03285,0.03285,0.03285
2018-11-11 18:13:14,0.03285,0.308196,0.03285,0.03285,0.03285,0.03285
2018-11-11 18:13:16,0.03285,0.479531,0.03285,0.03285,0.03285,0.03285


In [13]:
# Testing addition of moving averages

# import cufflinks as cf
# from plotly.graph_objs import FigureWidget
# cf.set_config_file(world_readable=True,offline=False)

# cf.datagen.lines(1,500).ta_plot(study='sma',periods=[13,21,55],title='Simple Moving Averages')

In [14]:
INCREASING_COLOR = '#17BECF'
DECREASING_COLOR = '#7F7F7F'

colors=[]
for i in range(len(df_prices_modified.price_close)):
    if i != 0:
        if df_prices_modified.price_close[i] > df_prices_modified.price_close[i-1]:
            colors.append(INCREASING_COLOR)
        else:
            colors.append(DECREASING_COLOR)
    else:
        colors.append(DECREASING_COLOR)

def movingaverage(interval, window_size=10):
    window = np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

In [15]:
import plotly
import plotly.plotly as py
from plotly.tools import FigureFactory as FF
import plotly.graph_objs as go

data1 = [ dict(
    type = 'candlestick',
    open=df_prices_modified.price_start,
    high=df_prices_modified.price_max,
    low=df_prices_modified.price_min,
    close=df_prices_modified.price_close,
    x = df_prices_modified.index,
    yaxis = 'y2',
    name = 'GS',
    increasing = dict( line = dict( color = INCREASING_COLOR ) ),
    decreasing = dict( line = dict( color = DECREASING_COLOR ) ),
) ]

layout=dict()
fig = dict(data=data1, layout=layout)

fig['layout']['plot_bgcolor'] = 'rgb(250, 250, 250)'
fig['layout']['xaxis'] = dict( rangeselector = dict( visible = True ) )
fig['layout']['yaxis'] = dict( domain = [0, 0.2], showticklabels = False )
fig['layout']['yaxis2'] = dict( domain = [0.2, 0.8] )
fig['layout']['legend'] = dict( orientation = 'h', y=0.9, x=0.3, yanchor='bottom' )
fig['layout']['margin'] = dict( t=40, b=40, r=40, l=40 )
fig['layout']['title'] = "{} to {} Comparison on {} based on {} intervals".format(pair1,pair2,exchange,time_interval)



# trace = go.Candlestick(x=df_prices_modified.index,
#                        open=df_prices_modified.price_start,
#                        high=df_prices_modified.price_max,
#                        low=df_prices_modified.price_min,
#                        close=df_prices_modified.price_close,
#                        increasing = dict( line = dict( color = INCREASING_COLOR ) ), 
#                        decreasing = dict( line = dict( color = DECREASING_COLOR ) ),
#                       )
# data1 = [trace]

add_line = go.Scatter(
    x=df_prices_modified.index,
    y=df_prices_modified.price_start,
    name= 'Open Averages {}-{}'.format(pair1, pair2),
    line=go.Line(color='black')
)

# adding volume

fig['data'].append( dict(x=df_prices_modified.index, y=df_prices_modified.volume,                         
                         marker=dict( color=colors ),
                         type='bar', yaxis='y', name='Volume' ) )

# adding moving averages

mv_x = list(df_prices_modified.index)
mv_y = movingaverage(df_prices_modified.price_close)

# Clip the ends
mv_x = mv_x[5:-5]
mv_y = mv_y[5:-5]

fig['data'].append( dict( x=mv_x, y=mv_y, type='scatter', mode='lines', 
                         line = dict( width = 1 ),
                         marker = dict( color = '#E377C2' ),
                         yaxis = 'y2', name='Moving Average' ) )


layout = go.Layout(
    autosize=False,
    width=1000,
    height=700,
#     xaxis = dict(
#         rangeslider = dict(
#             visible = False
#         )
#     ),
    title="{} to {} Comparison on {} based on {} intervals".format(pair1,pair2,exchange,time_interval)
)

# py.iplot(fig, filename='candlestick_trace')
py.iplot( fig, filename = 'candlestick-test-3', validate = False )


# Create Candlestick
# fig = plotly.figure_factory.create_candlestick(df_prices_modified.price_start, 
#                             df_prices_modified.price_max, 
#                             df_prices_modified.price_min, 
#                             df_prices_modified.price_close, 
#                             dates=df_prices_modified.index)

# Create Line of open values


# add_line = Scatter(
#     x=df_prices_modified.index,
#     y=df_prices_modified.price_start,
#     name= 'Open Averages {}-{}'.format(pair1, pair2),
#     line=Line(color='black')
# )



# fig['data'].extend([add_line])
# py.iplot(fig, filename='simple_candlestick')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:372: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


